In [1]:
print("ok")

ok


In [20]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [21]:
from pinecone import Pinecone

pc = Pinecone()



In [22]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [23]:
extracted_data = load_pdf("data/")

In [25]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [26]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [27]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [28]:
embeddings = download_hugging_face_embeddings()

In [29]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [30]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [45]:
import os
import fitz  # PyMuPDF
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

# Function to extract text from PDF files in a directory
def extract_text_from_pdfs(directory):
    pdf_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.pdf')]
    extracted_text = []
    for file_path in pdf_files:
        # Open the PDF file
        with fitz.open(file_path) as pdf_doc:
            text = ""
            # Iterate through each page
            for page_num in range(len(pdf_doc)):
                page = pdf_doc.load_page(page_num)
                text += page.get_text()
            extracted_text.append(text)
    return extracted_text

# Function to generate embeddings for text chunks
def generate_embeddings(text_chunks, model):
    embeddings = []
    for chunk in text_chunks:
        # Generate embedding for each text chunk
        embedding = model.encode(chunk)
        embeddings.append(embedding)
    return embeddings

try:
    # Initialize Pinecone client with API Key from environment variable
    pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))

    # Assuming 'data/' is the directory containing PDF files
    pdf_directory = 'data/'

    # Extract text from PDF files in the directory
    text_chunks = extract_text_from_pdfs(pdf_directory)

    # Load Hugging Face embeddings model
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    # Generate embeddings for each text chunk
    embeddings = generate_embeddings(text_chunks, model)

    # Index name
    index_name = "medical-chatbot"

    # Dimensionality of the embeddings
    dimension = 384

    # Index specification
    spec = None  # You can specify the index specification according to your requirements

    # Create an index with the specified name, dimension, and spec
    pc.create_index(index_name, dimension=dimension, spec=spec)

    # Upsert embeddings into the index
    pc.upsert(index_name, embeddings)

    # Now you can perform operations on the created index like querying, updating, etc.
except Exception as e:
    print("An error occurred:", e)


An error occurred: spec must be of type dict, ServerlessSpec, or PodSpec


In [49]:
import os
import fitz  # PyMuPDF
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

# Function to extract text from PDF files in a directory
def extract_text_from_pdfs(directory):
    pdf_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.pdf')]
    extracted_text = []
    for file_path in pdf_files:
        # Open the PDF file
        with fitz.open(file_path) as pdf_doc:
            text = ""
            # Iterate through each page
            for page_num in range(len(pdf_doc)):
                page = pdf_doc.load_page(page_num)
                text += page.get_text()
            extracted_text.append(text)
    return extracted_text

# Function to generate embeddings for text chunks
def generate_embeddings(text_chunks, model):
    embeddings = []
    for chunk in text_chunks:
        # Generate embedding for each text chunk
        embedding = model.encode(chunk)
        embeddings.append(embedding)
    return embeddings

try:
    # Initialize Pinecone client with API Key from environment variable
    pc = Pinecone(api_key=os.environ.get('5d30ff0e-6bc6-41f4-8f3e-5654d3b4a131'))

    # Assuming 'data/' is the directory containing PDF files
    pdf_directory = 'data/'

    # Extract text from PDF files in the directory
    text_chunks = extract_text_from_pdfs(pdf_directory)

    # Load Hugging Face embeddings model
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    # Generate embeddings for each text chunk
    embeddings = generate_embeddings(text_chunks, model)

    # Index name
    index_name = "medical-chatbot"

    # Dimensionality of the embeddings
    dimension = 384

    # Index specification (example: serverless index in AWS us-west-2 region)
    spec = {"cloud": "aws", "region": "Iowa (us-central1)"}

    # Create an index with the specified name, dimension, and spec
    pc.create_index(index_name, dimension=dimension, spec=spec)

    # Upsert embeddings into the index
    pc.upsert(index_name, embeddings)

    # Now you can perform operations on the created index like querying, updating, etc.
except Exception as e:
    print("An error occurred:", e)


An error occurred: (422)
Reason: unknown
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': 'bc72cdefda94a7605dd108a685986558', 'Date': 'Wed, 27 Mar 2024 19:44:14 GMT', 'Server': 'Google Frontend', 'Content-Length': '139', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Failed to deserialize the JSON body into the target type: spec: unknown variant `cloud`, expected `pod` or `serverless` at line 1 column 62

